# Sibilant + stop Deaspiration Phenomenon Selection

Here we want to work out how we can select only those instances (words) with only target seqs. But one problem is that we don't have teh exact recording files on that granularity level. We only have cut words and cut phones. But our target is something like two or three phones. This is a problem. 

However, considering that our target is not very long, I am thinking of finding all valid instances and integrate them into recordings. Then each time we train, read from the integrated recordings. 

In [214]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import torch
import torchaudio
from torch import nn
import torch.nn.functional as F
from torch import optim
from torch.utils.data import DataLoader, random_split
import matplotlib.pyplot as plt
import random
from IPython.display import Audio
from tqdm import tqdm
import seaborn as sns
import pandas as pd
from collections import Counter
from sklearn.manifold import TSNE   # one type of clustering
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.metrics import adjusted_rand_score
from sklearn.metrics import homogeneity_score, completeness_score, v_measure_score
from itertools import combinations
from sklearn.decomposition import PCA
from scipy.linalg import block_diag
import pickle
from scipy import stats
from model_padding import generate_mask_from_lengths_mat, mask_it
from paths import *
from misc_my_utils import *
from model_loss import *
from model_model import CTCPredNetV1 as TheLearner
from model_dataset import WordDatasetPath as ThisDataset
from model_dataset import Normalizer, DeNormalizer, TokenMap
from model_dataset import MelSpecTransformDB as TheTransform
from model_dataset import DS_Tools
from reshandler import DictResHandler
from misc_progress_bar import draw_progress_bar
from test_bnd_detect_tools import *
from misc_tools import PathUtils as PU
from misc_tools import AudioCut, ARPABET
import re

In [215]:
rec_dir = train_cut_word_
train_guide_path = os.path.join(src_, "guide_train.csv")
valid_guide_path = os.path.join(src_, "guide_validation.csv")
test_guide_path = os.path.join(src_, "guide_test.csv")

In [216]:
# read in guide file
guide_file = pd.read_csv(valid_guide_path)
# filtering out is not necessary, since we only include wuid for encoded words
guide_file = guide_file[~guide_file["segment_nostress"].isin(["sil", "sp", "spn"])]

In [217]:
# words_guide = guide_file.groupby('wuid').apply(lambda x: ([row["segment"] for index, row in x.iterrows()]).tolist()
words_guide_str = guide_file.groupby('wuid').apply(lambda x: (" ".join([row["segment"] for index, row in x.iterrows()]), x["wuid"].iloc[0])).tolist()

In [218]:
def regex_span_to_list_indices(phoneme_str, pattern):
    # Split the string into a list of phonemes
    phonemes = phoneme_str.split()
    # Calculate the cumulative lengths including spaces (add 1 for each space)
    cumulative_lengths = [0]  # Start with 0 for the first phoneme
    for phoneme in phonemes:
        # Add the length of the current phoneme and a space (except for the last one)
        cumulative_lengths.append(cumulative_lengths[-1] + len(phoneme) + 1)
    # Find all matches using re.finditer
    matches = list(re.finditer(pattern, phoneme_str))
    # Map regex span indices to phoneme list indices
    match_indices = []
    for match in matches:
        start, end = match.span()
        # Find the phoneme list index corresponding to the start of the match
        list_start = next(i for i, length in enumerate(cumulative_lengths) if length > start) - 1
        # Find the phoneme list index corresponding to the end of the match (subtract 1 because end is exclusive)
        list_end = next(i for i, length in enumerate(cumulative_lengths) if length >= end) - 1
        match_indices.append((list_start, list_end))
    return match_indices

In [233]:
stop_pattern = '(?!S) [PK] (?!R)'
sibstop_pattern = 'S [PTK] (?!R)'

In [234]:
Xstop_indices = [name for i, (word, name) in enumerate(words_guide_str) if re.search(stop_pattern, word)]
sibstop_indices = [name for i, (word, name) in enumerate(words_guide_str) if re.search(sibstop_pattern, word)]
sibstop_subidx = [regex_span_to_list_indices(word, sibstop_pattern) for i, (word, name) in enumerate(words_guide_str) if re.search(sibstop_pattern, word)]
Xstop_subidx = [regex_span_to_list_indices(word, stop_pattern) for i, (word, name) in enumerate(words_guide_str) if re.search(stop_pattern, word)]

Here we have the risk of including /t/ for ST but excluding it in XT. 

In [235]:
len(Xstop_indices), len(sibstop_indices)

(6325, 2606)

(10729, 2606)

In [237]:
def generate_table(df, name_list, target_idx_list=None): 
    sibilant_list = []
    sibilant_path = []
    sibilant_startTime = []
    sibilant_endTime = []
    stop_list = []
    stop_path = []
    stop_startTime = []
    stop_endTime = []
    speaker_list = []
    wuid_list = []
    if target_idx_list is None:
        col_dict = {
            "segment_nostress": "stop", 
            "startTime": "stop_startTime", 
            "endTime": "stop_endTime", 
            "phone_path": "stop_path", 
            "speaker": "speaker", 
            "wuid": "wuid"
        }
        word_phonemes = df[df["wuid"].isin(name_list)]
        word_phonemes["wuid_g"] = word_phonemes["wuid"]
        outdf = word_phonemes.groupby("wuid_g").first().reset_index()
        outdf = outdf[col_dict.keys()]
        outdf = outdf.rename(columns=col_dict)
    else:
        for name, target_idx in zip(name_list, target_idx_list): 
            # this is one word, there might be multiple matching cases
            word_phonemes = df[df["wuid"] == name]
            for target in target_idx: 
                target = [i + 1 for i in target]
                target_phonemes = word_phonemes[word_phonemes["in_id"].isin(target)]
                # print(target_phonemes)
                sib = target_phonemes.iloc[0]
                stop = target_phonemes.iloc[1]
                sibilant_list.append(sib["segment_nostress"])
                sibilant_path.append(sib["phone_path"])
                sibilant_startTime.append(sib["startTime"])
                sibilant_endTime.append(sib["endTime"])

                stop_list.append(stop["segment_nostress"])
                stop_path.append(stop["phone_path"])
                stop_startTime.append(stop["startTime"])
                stop_endTime.append(stop["endTime"])

                speaker_list.append(stop["speaker"])
                wuid_list.append(name)
        out_dict = {
            "pre": sibilant_list, 
            "stop": stop_list, 
            "pre_path": sibilant_path, 
            "stop_path": stop_path, 
            "pre_startTime": sibilant_startTime, 
            "pre_endTime": sibilant_endTime, 
            "stop_startTime": stop_startTime,
            "stop_endTime": stop_endTime,
            "speaker": speaker_list,
            "wuid": wuid_list
        }
        outdf = pd.DataFrame(out_dict)
    return outdf

In [226]:
sibstopsdf = generate_table(guide_file, sibstop_indices, sibstop_subidx)

In [242]:
rename_sibstopsdf = sibstopsdf.rename(columns={"sibilant": "pre", "sibilant_path": "pre_path", "sibilant_startTime": "pre_startTime", "sibilant_endTime": "pre_endTime"})

In [243]:
stopsdf = generate_table(guide_file, Xstop_indices, Xstop_subidx)

In [202]:
result_df = pd.concat([sibstopsdf, stopsdf], ignore_index=True, sort=False)

In [244]:
stopsdf

,pre,stop,pre_path,stop_path,pre_startTime,pre_endTime,stop_startTime,stop_endTime,speaker,wuid
0,AE,P,103/1240/0000/103-1240-0000-0002.flac,103/1240/0000/103-1240-0000-0003.flac,0.57,0.63,0.63,0.70,103,103-1240-0000-0000
1,IH,P,103/1240/0000/103-1240-0000-0077.flac,103/1240/0000/103-1240-0000-0078.flac,8.82,8.88,8.88,9.00,103,103-1240-0000-0017
2,AA,P,103/1240/0000/103-1240-0000-0125.flac,103/1240/0000/103-1240-0000-0126.flac,12.54,12.62,12.62,12.71,103,103-1240-0000-0028
3,IH,P,103/1240/0001/103-1240-0001-0052.flac,103/1240/0001/103-1240-0001-0053.flac,4.54,4.59,4.59,4.65,103,103-1240-0001-0016
4,AH,K,103/1240/0001/103-1240-0001-0069.flac,103/1240/0001/103-1240-0001-0070.flac,5.53,5.58,5.58,5.63,103,103-1240-0001-0020
...,...,...,...,...,...,...,...,...,...,...
6790,ER,P,909/131045/0042/909-131045-0042-0162.flac,909/131045/0042/909-131045-0042-0163.flac,12.97,13.01,13.01,13.13,909,909-131045-0042-0037
6791,EH,K,909/131045/0043/909-131045-0043-0036.flac,909/131045/0043/909-131045-0043-0037.flac,2.87,2.97,2.97,3.04,909,909-131045-0043-0007
6792,EH,K,909/131045/0043/909-131045-0043-0074.flac,909/131045/0043/909-131045-0043-0075.flac,7.25,7.38,7.38,7.48,909,909-131045-0043-0014
6793,IY,P,909/131045/0043/909-131045-0043-0112.flac,909/131045/0043/909-131045-0043-0113.flac,10.60,10.70,10.70,10.77,909,909-131045-0043-0023


In [206]:
result_df["phi_type"] = np.where(result_df["sibilant"].isna(), "T", "ST")

In [212]:
tg = result_df[result_df["phi_type"] == "T"]
stg = result_df[result_df["phi_type"] == "ST"]

In [213]:
tg.to_csv(os.path.join(src_, "phi-T-guide.csv"), index=False)
stg.to_csv(os.path.join(src_, "phi-ST-guide.csv"), index=False)

# Generate training and testing sets

In [245]:
phi_T_guid_path = os.path.join(src_, "phi-T-guide.csv")
phi_ST_guid_path = os.path.join(src_, "phi-ST-guide.csv")

In [247]:
# We do the random selection here
t_guide = pd.read_csv(phi_T_guid_path)
st_guide = pd.read_csv(phi_ST_guid_path)
# sampled_t_guide = t_guide.sample(n=len(t_guide))
# all_guide = pd.concat([st_guide, sampled_t_guide], ignore_index=True, sort=False)

In [263]:
def random_sample_by_speaker(tg, stg, valid_proportion=0.2): 
    # sample speakers, tg and stg use the same, because they must have the same speaker choice 
    speakerlist = stg["speaker"].unique()
    valid_size = int(len(speakerlist) * valid_proportion)
    sampled_indices = np.random.choice(len(speakerlist), size=valid_size, replace=False)
    valid_speakers = speakerlist[sampled_indices]

    t_stg = stg[~stg["speaker"].isin(valid_speakers)]
    v_stg = stg[stg["speaker"].isin(valid_speakers)]

    t_tg = tg[~tg["speaker"].isin(valid_speakers)]
    v_tg = tg[tg["speaker"].isin(valid_speakers)]

    t_s_tg = t_tg.sample(n=len(t_stg))
    v_s_tg = v_tg.sample(n=len(v_stg))

    return (t_stg, v_stg), (t_s_tg, v_s_tg)

In [264]:
(training_st, valid_st), (training_t, valid_t) = random_sample_by_speaker(t_guide, st_guide)

In [ ]:
len(training_st), len(valid_st), len(training_t), len(valid_t)